In [1]:
import pandas as pd
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

In [20]:
df = pd.read_csv('../ROOT_DATA/uvwNuanceMu.csv', header=None, names = ['isTrueTrack','sequence','Nuance', 'pdg', 'completeness', 'purity', 'xu', 'zu', 'xv', 'zv', 'x', 'z' ])
df= df.drop(columns=['xv', 'zv', 'xu','zu'])

In [22]:
def sequence2event(sequence):
    current=sequence[0]
    event=[0]
    for i in sequence[1:]:
        if i==current:
            event.append(event[-1])
        else:
            current=i
            event.append(event[-1]+1)
    return event
df['event']=sequence2event(df.sequence)
df=df.drop(columns=['sequence'])
df.head()

,isTrueTrack,Nuance,pdg,completeness,purity,x,z,event
0,1,1001,13,0.978231,0.998611,102.367 102.453 103.276 104.281 105.02 105.84 ...,293.35 293.65 293.95 294.25 294.55 294.85 295....,0
1,1,1001,2212,0.741379,0.988506,102 101.839 101.675 101.496 101.358 101.156 10...,293.95 294.25 294.55 294.85 295.15 295.45 295....,0
2,1,1001,13,1.000000,1.000000,216.457 216.239 216.036 215.857 215.676 215.50...,535.45 535.75 536.05 536.35 536.65 536.95 537....,1
3,1,1001,13,0.948052,0.533333,65.5685 65.612 65.6608 65.7068 65.7643 65.8244...,131.35 131.65 131.95 132.25 132.55 132.85 133....,2
4,1,1001,13,0.050325,1.000000,64.0988 64.2145 64.2907 64.3714 64.4339 64.542...,125.05 125.35 125.65 125.95 126.25 126.55 126....,2


In [35]:
pdg_dict={'p':2212,'mu-':13, 'gamma':22, 'pi0':11,'pi+':211, 'e':11 }

In [102]:
nuance=1004
nuancedf=df[df['Nuance']==nuance]
nuancedf.head()

,isTrueTrack,Nuance,pdg,completeness,purity,x,z,event
8,0,1004,22,0.721154,1.000000,20.7565 20.7275 20.9535 21.1775 21.3639 21.504...,646.75 647.05 647.35 647.65 647.95 648.25 648....,5
9,1,1004,13,1.000000,1.000000,44.7719 44.7762 44.7473 44.7386 44.7131 44.698...,696.55 696.85 697.15 697.45 697.75 698.05 698....,5
10,0,1004,22,0.392593,1.000000,43.1104 42.7561 42.1619 40.7982 41.5836 41.120...,697.45 697.75 698.05 698.35 698.35 698.65 698....,5
181,1,1004,2212,0.995781,0.991597,193.532 193.537 193.596 193.676 193.753 193.83...,795.55 795.85 796.15 796.45 796.75 797.05 797....,84
182,1,1004,13,0.992780,0.996377,235.189 235.025 234.632 234.117 233.791 233.45...,762.25 762.55 762.85 763.15 763.45 763.75 764....,84


In [103]:
def get_freq_events(df,freq):
    new_index=[]
    for i in sorted(set(df.event)):
        if len(df[df.event==i])==freq:
            new_index.append(i)
    return new_index
freq_events = get_freq_events(nuancedf,4)

In [104]:
def get_correct_events(df, freq_events):
    new_index=[]
    for i in freq_events:
        if len(df[(df.event==i) & (df.pdg==pdg_dict['p'])])==1 and len(df[(df.event==i) & (df.pdg==pdg_dict['mu-'])])==1 and len(df[(df.event==i) & (df.pdg==pdg_dict['gamma'])])==2:
            new_index.append(i)
    return new_index
final_events = get_correct_events(nuancedf, freq_events)
    

In [105]:
final_df= nuancedf.loc[nuancedf['event'].isin(final_events)]
final_df.head()

,isTrueTrack,Nuance,pdg,completeness,purity,x,z,event
336,0,1004,22,1.000000,0.971888,255.001 239.437 254.815 239.635 254.425 240.19...,66.25 66.55 66.55 66.85 66.85 67.15 67.15 67.4...,149
337,0,1004,22,0.942149,1.000000,251.16 252.264 250.994 250.228 249.548 249 248...,57.85 57.85 58.15 58.45 58.75 59.05 59.35 59.6...,149
338,1,1004,13,0.992584,0.983607,231.146 231.145 231.126 231.106 231.075 231.03...,62.95 63.25 63.55 63.85 64.15 64.45 64.75 65.0...,149
339,1,1004,2212,0.753191,1.000000,230.532 230.499 230.459 230.405 230.369 230.33...,70.45 70.75 71.05 71.35 71.65 71.95 72.25 72.5...,149
525,0,1004,22,0.969072,1.000000,251.405 251.392 251.351 228.146 228.149 228.08...,355.75 356.05 356.35 362.05 362.35 362.65 362....,232


373

In [107]:
final_df.to_csv('../ROOT_DATA/1004nuance_mu_p_gamma_gamma.csv')

1492